# Imports 

In [22]:
import pandas as pd
import numpy as np
import pickle
import time

In [23]:
startTime = time.time()

# Get Data

In [24]:
filename = '../../002_Data/Release_3/data_combined.pickle'
infile = open(filename,'rb')
df = pickle.load(infile)
infile.close()
df.head()

,SPA_PER_ID,SPA_PREM_ID,ARREARSMONTH,SPA_ACCT_ID,RES_EL_CUR120_DAYS,RES_EL_CUR22_DAYS,RES_EL_CUR30_DAYS,RES_EL_CUR60_DAYS,RES_EL_CUR90_DAYS,RES_EL_CUR_BAL_AMT,...,TOTAL_30_DAYS_AMT,TOTAL_60_DAYS_AMT,TOTAL_90_DAYS_AMT,TOTAL_CUR_BALANCE,CMIS_DB_ENTRY,MO_AWAY,WITHIN_6_MO_PRIOR_LAST_DATA,LAST_MO_W_DATA,NUM_PREM_FOR_PER,NUM_PER_FOR_PREM
0,1,59971,59,249321,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,False,NaN,False,False,1,2
1,1,59971,60,249321,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,False,NaN,False,False,1,2
2,3,98612,0,171648,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,103.40,False,NaN,False,False,1,1
3,3,98612,1,171648,0.0,65.26,0.0,0.0,0.0,65.26,...,0.0,0.0,0.0,307.71,False,NaN,False,False,1,1
4,3,98612,2,171648,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,105.80,False,NaN,False,False,1,1


In [25]:
df.columns.to_list()

['SPA_PER_ID',
 'SPA_PREM_ID',
 'ARREARSMONTH',
 'SPA_ACCT_ID',
 'RES_EL_CUR120_DAYS',
 'RES_EL_CUR22_DAYS',
 'RES_EL_CUR30_DAYS',
 'RES_EL_CUR60_DAYS',
 'RES_EL_CUR90_DAYS',
 'RES_EL_CUR_BAL_AMT',
 'RES_EL_OVER_120_DAYS',
 'RES_GAS_CUR120_DAYS',
 'RES_GAS_CUR22_DAYS',
 'RES_GAS_CUR30_DAYS',
 'RES_GAS_CUR60_DAYS',
 'RES_GAS_CUR90_DAYS',
 'RES_GAS_CUR_BAL_AMT',
 'RES_GAS_OVER_120_DAYS',
 'BREAK_ARRANGEMENT',
 'BREAK_PAY_PLAN',
 'CALL_OUT',
 'CALL_OUT_MANUAL',
 'DUE_DATE',
 'FINAL_NOTICE',
 'PAST_DUE',
 'SEVERANCE_ELECTRIC',
 'SEVERANCE_GAS',
 'CITY_TOT_DUE',
 'CITY_30_DAYS_PAST_DUE_AMT',
 'CITY_60_DAYS_PAST_DUE_AMT',
 'CITY_90_DAYS_PAST_DUE_AMT',
 'COVID_REMINDER',
 'CMIS_MATCH',
 'APARTMENT',
 'ENROLL_DATE',
 'HAS_COTENANT',
 'BLOCKGROUP_GEOID',
 'POSTAL',
 'AGG_INCOME_GEO',
 'NO_EARNINGS_GEO',
 'BELOW_POVERTY_LVL_GEO',
 'SNAP_GEO',
 'ABOVE_GRD7_GEO',
 'PUBLIC_ASSIST_GEO',
 'AVISTA_CUR120_DAYS',
 'AVISTA_OVER_120_DAYS',
 'AVISTA_CUR22_DAYS',
 'AVISTA_CUR30_DAYS',
 'AVISTA_CUR60_DAYS',


In [26]:
to_drop = [
    # Billing
    'RES_EL_CUR120_DAYS',
    'RES_EL_CUR22_DAYS',
    'RES_EL_CUR30_DAYS',
    'RES_EL_CUR60_DAYS',
    'RES_EL_CUR90_DAYS',
    'RES_EL_CUR_BAL_AMT',
    'RES_EL_OVER_120_DAYS',
    'RES_GAS_CUR120_DAYS',
    'RES_GAS_CUR22_DAYS',
    'RES_GAS_CUR30_DAYS',
    'RES_GAS_CUR60_DAYS',
    'RES_GAS_CUR90_DAYS',
    'RES_GAS_CUR_BAL_AMT',
    'RES_GAS_OVER_120_DAYS',
    'CITY_TOT_DUE',
    'CITY_30_DAYS_PAST_DUE_AMT',
    'CITY_60_DAYS_PAST_DUE_AMT',
    'CITY_90_DAYS_PAST_DUE_AMT',
    'AVISTA_CUR120_DAYS',
    'AVISTA_OVER_120_DAYS',
    'AVISTA_CUR22_DAYS',
    'AVISTA_CUR30_DAYS',
    'AVISTA_CUR60_DAYS',
    'AVISTA_CUR90_DAYS',
    'AVISTA_CUR_BAL_AMT',
    'TOTAL_30_DAYS_AMT',
    'TOTAL_60_DAYS_AMT',
    'TOTAL_90_DAYS_AMT',
    # Only 2020
    'COVID_REMINDER',
    # Don't want to predict on these
    'SPA_PREM_ID',
    'ARREARSMONTH',
    'SPA_ACCT_ID',
    'POSTAL',
    'BLOCKGROUP_GEOID',
    'ENROLL_DATE',
    'MO_AWAY'
]
df.drop(to_drop, axis=1, inplace=True)

## Choose Predictor Variable

In [27]:
event_col = 0

event_cols = [
    'CMIS_MATCH',
    "CMIS_DB_ENTRY",
    'LAST_MO_W_DATA',
    "WITHIN_6_MO_PRIOR_LAST_DATA"
]
event_col = event_cols[event_col]
event_cols.remove(event_col)
df = df.drop(event_cols, axis=1).drop_duplicates()

id_col = "SPA_PER_ID"

df = df.astype('float')

df.columns.to_list()

['SPA_PER_ID',
 'BREAK_ARRANGEMENT',
 'BREAK_PAY_PLAN',
 'CALL_OUT',
 'CALL_OUT_MANUAL',
 'DUE_DATE',
 'FINAL_NOTICE',
 'PAST_DUE',
 'SEVERANCE_ELECTRIC',
 'SEVERANCE_GAS',
 'CMIS_MATCH',
 'APARTMENT',
 'HAS_COTENANT',
 'AGG_INCOME_GEO',
 'NO_EARNINGS_GEO',
 'BELOW_POVERTY_LVL_GEO',
 'SNAP_GEO',
 'ABOVE_GRD7_GEO',
 'PUBLIC_ASSIST_GEO',
 'TOTAL_CUR_BALANCE',
 'NUM_PREM_FOR_PER',
 'NUM_PER_FOR_PREM']

## Normalize Data

In [62]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from custom_methods import model_eval
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

df_test, df_train = model_eval.split_on_people(df, id_col)

sc = StandardScaler()
df_train.update(sc.fit_transform(df_train.drop([event_col, id_col], axis=1)))

df_test.update(sc.transform(df_test.drop([event_col, id_col], axis=1)))
# Undersample N Cases
print(f"N Cases: {len(df_train[df_train[event_col] == 0.0])}, P Cases: {len(df_train[df_train[event_col] == 1.0])}")
rus = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_res, y_res = rus.fit_sample(X=df_train.drop([event_col, id_col], axis=1), y=df_train[event_col])
print(f"Cases: {Counter(y_res)}")

#class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=df_train[event_col].unique(), y=df_train[event_col])))

N Cases: 1004085, P Cases: 2180
Cases: Counter({0.0: 2180, 1.0: 2180})


# Build Model

In [29]:
from keras.callbacks import EarlyStopping

In [70]:
# Source: https://analyticsindiamag.com/how-to-create-your-first-artificial-neural-network-in-python/
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD

from custom_methods import model_eval

model = Sequential()

callback = EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    patience=2,
    restore_best_weights=True
)

# Create input layer + first hidden layer
model.add(Dense(input_dim = len(df.drop([event_col, id_col], axis=1).columns), units = 10, activation='tanh', kernel_initializer='uniform'))
# Add second hidden layer
model.add(Dense(units = 20, activation='relu', kernel_initializer='uniform'))
# Create output layer
model.add(Dense(units = 1, activation='sigmoid', kernel_initializer='uniform'))

# Compile ANN
model.compile(
    optimizer=SGD(), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

# Fit ANN
#model.fit(df_train.drop([event_col, id_col], axis=1), df_train[event_col], batch_size=10, epochs=5)

In [71]:
#df_train, df_test = model_eval.split_on_people(df, id_col='SPA_PER_ID')
model.fit(
    x=X_res, #df_train.drop([event_col, id_col], axis=1), 
    y=y_res, #df_train[event_col], 
    batch_size=10, 
    epochs=10,
    #class_weight=class_weights,
    callbacks=[callback]
)

Epoch 1/10
436/436 [==============================] - 1s 935us/step - loss: 0.6932 - accuracy: 0.5021
Epoch 2/10
436/436 [==============================] - 0s 736us/step - loss: 0.6933 - accuracy: 0.4881
Epoch 3/10
436/436 [==============================] - 0s 736us/step - loss: 0.6933 - accuracy: 0.4947


In [72]:
predictions = model.predict(x=df_test.drop([event_col, id_col], axis=1))
predictions = pd.Series(np.transpose(predictions)[0])
actuals = df_test[event_col].reset_index()[event_col]

In [73]:
predictions.value_counts()

0.501543    2046008
dtype: int64

In [33]:
results = model_eval.get_model_metrics(predictions, actuals.astype('bool'))
results

Threshold_Value Number Predicted Actual Number  \
Threshold Case                                                      
0.01      Positive             NaN              NaN           NaN   
          Negative             NaN              NaN           NaN   
0.05      Positive             NaN              NaN           NaN   
          Negative             NaN              NaN           NaN   
0.10      Positive             NaN              NaN           NaN   
          Negative             NaN              NaN           NaN   

                   False P/N Rate Precision Recall  f-1 Custom  
Threshold Case                                                  
0.01      Positive            NaN       NaN    NaN  NaN    NaN  
          Negative            NaN       NaN    NaN  NaN    NaN  
0.05      Positive            NaN       NaN    NaN  NaN    NaN  
          Negative            NaN       NaN    NaN  NaN    NaN  
0.10      Positive            NaN       NaN    NaN  NaN    NaN  
          Negative            NaN       NaN    NaN  NaN    NaN

In [34]:
'''
k = 10
results, summary = model_eval.k_fold_models(
    df=df, 
    event_col='CMIS_MATCH',
    id_col='SPA_PER_ID', 
    k=10, 
    model=model, 
    cutoffs=[0.05, 0.10, 0.15]
)
summary
'''

"\nk = 10\nresults, summary = model_eval.k_fold_models(\n    df=df, \n    event_col='CMIS_MATCH',\n    id_col='SPA_PER_ID', \n    k=10, \n    model=model, \n    cutoffs=[0.05, 0.10, 0.15]\n)\nsummary\n"

## Print Time

In [35]:
from custom_methods.calc_time import calc_time_from_sec

calc_time_from_sec(time.time()-startTime)

hours:minutes:seconds = 0:6:20.60689878463745


## Evaluate Model